In [0]:
import os
from google.colab import drive
drive.mount('/content/gdrive/')
os.chdir("./gdrive/My Drive/NLU/Projects/project 1/rok/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
import tensorflow as tf
import numpy as np

print("tf_version:\t" + tf.__version__)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"

from models import ModelA

# DIRECTORIES
DATA_DIR = "./data/"
RESULTS_DIR = "./results/"
WORD_EMBEDDINGS_FILE = "wordembeddings-dim100.word2vec"
SENTENCES_TRAIN_FILE = "sentences.train"
SENTENCES_TEST_FILE = "sentences_test.txt"
SENTENCES_EVAL_FILE = "sentences.eval"
SENTENCES_CONTINUATION_FILE = "sentences.continuation"

# LANGUAGE MODEL PARAMETERS
EMBEDDING_DIM = 100
STATE_DIM = 512
VOCABULARY_SIZE = 20000
SENT_DIM = 30

# RNN PARAMETERS
BATCH_SIZE = 64
LEARNING_RATE = 0.001
MAX_GRAD_NORM = 5.0
NUM_EPOCHS = 1
KEEP_PROBS = 0.5

tf_version:	1.13.1


In [0]:
EXPERIMENT = "A"

tf.reset_default_graph()
tf.set_random_seed(12345)
np.random.seed(12345)

# Train
model = ModelA(vocabulary_size=VOCABULARY_SIZE, embedding_dim=EMBEDDING_DIM, state_dim=STATE_DIM, sent_dim=SENT_DIM,
               initializer=tf.contrib.layers.xavier_initializer(), pad_idx=180, tf_graph=None)

saver = tf.train.Saver()

with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    # TRAINING
    session.run(model.iterator_op, {model.sentences_file: RESULTS_DIR + "X_train.ids"})
    batch_count = 0
    total_batch = 500
    while True:
        try:
            batch_loss, batch_perplexity, _ = session.run([model.loss, model.perplexity, model.optimize_op])
            epoch = 1
            if batch_count % 100 == 0:
                print("epoch: {}/{:<6}batch: {:>5}/{:<10}loss = {:<13.2f}perp = {:<13.2f}".format(epoch, NUM_EPOCHS,
                                                                                                  batch_count + 1,
                                                                                                  total_batch,
                                                                                                  batch_loss,
                                                                                                  batch_perplexity))

            batch_count += 1
            if batch_count > total_batch:
                break
        except tf.errors.OutOfRangeError:
            break

    save_path = saver.save(session, MODEL_DIR + "/experiment" + EXPERIMENT + 
                           "/experiment" + EXPERIMENT + ".ckpt")
    print("Model saved in path: %s" % save_path)
            
    # EVALUATION
    session.run(model.iterator_op, {model.sentences_file: RESULTS_DIR + "X_eval.ids"})
    eval_perplexities = np.array([], dtype=np.float32)
    batch_count = 0
    while True:
        try:
            batch_perplexities = session.run(model.perplexities)
            eval_perplexities = np.append(eval_perplexities, batch_perplexities)
            batch_count += 1
        except tf.errors.OutOfRangeError:
            break

with open(RESULTS_DIR + "groupXX.perplexity" + EXPERIMENT, "w") as f:
    for i in range(eval_perplexities.shape[0]):
        f.write("%0.3f" % eval_perplexities[i] + "\n")

epoch: 1/1     batch:     1/500       loss = 287.21       perp = 1406.07      
epoch: 1/1     batch:   101/500       loss = 85.02        perp = 82.61        
epoch: 1/1     batch:   201/500       loss = 82.52        perp = 53.76        
epoch: 1/1     batch:   301/500       loss = 72.43        perp = 37.39        
epoch: 1/1     batch:   401/500       loss = 60.28        perp = 21.85        
epoch: 1/1     batch:   501/500       loss = 72.70        perp = 41.78        
